<img src="./pictures/logo_sizinglab.png" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Design optimization and exploration of a multirotor drone

*Written by Marc Budinger (INSA Toulouse), Scott Delbecq (ISAE-SUPAERO) and Félix Pollet (ISAE-SUPAERO), Toulouse, France.*

The objective of this notebook is to use and modify the sizing code of the UAV to explore the design space.

## Modular sizing code

Here we use the sizing code but decomposed into several `ExplicitComponent` of [OpenMDAO](https://openmdao.org/newdocs/versions/latest/main.html). You might notice an improvement in computational cost and robustness compared to the previous sizing code that used a single `ExplicitComponent` to wrap all the sizing code.
The models are separated by components and located in different Python files `/models/openmdao`.

In [1]:
# Import libraries
import numpy as np
import openmdao.api as om

from models.openmdao.scenarios import Scenarios
from models.openmdao.propeller import Propeller
from models.openmdao.motor import Motor
from models.openmdao.battery import Battery
from models.openmdao.esc import ESC
from models.openmdao.frame import Frame
from models.openmdao.objectives import Objectives
from models.openmdao.constraints import Constraints

The inputs of the whole sizing code are stored in `dict`s that we be passed to the problem before the run. Here are the default values seperated by:
- specifications
- design assumtpions 
- design variable initialization

However, one of the objective of the notebook is to modify these input values.

In [2]:
# Specifications
specifications_default = {
    "M_pay": 50.0,  # [kg] load mass
    "a_to": 0.25 * 9.81,  # [m/s**2] acceleration
    "t_hov_spec": 25.0,  # [min] time of hover flight
    "MTOW": 360.0,  # [kg] maximal mass allowed
}

# Design assumptions
design_assumptions_default = {
    "N_arm" : 4,  # [-] number of arms
    "N_pro_arm" : 1,  # [-] number of propeller per arm (1 or 2)
    "M_bat_ref" : 0.329,  # [kg] mass
    "E_bat_ref" : 220.0 * 3600.0 * 0.329,  # [J]
    "C_bat_ref" : 5,  # [Ah] Capacity
    "I_bat_max_ref" : 50 * 5,  # [A] max discharge current
    "P_esc_ref" : 3108.0,  # [W] Power
    "M_esc_ref" : 0.115,  # [kg] Mass
    "T_nom_mot_ref" : 2.32,  # [N*m] rated torque
    "T_max_mot_ref" : 85.0 / 70.0 * 2.32,  # [N*m] max torque
    "R_mot_ref" : 0.03,  # [ohm] resistance
    "M_mot_ref" : 0.575,  # [kg] mass
    "K_T_ref" : 0.03,  # [N*m/A] torque coefficient
    "T_mot_fr_ref" : 0.03,  # [N*m] friction torque (zero load, nominal speed)
    "sigma_max" : (
        280e6 / 4.0
    ),  # [Pa] Composite max stress (2 reduction for dynamic, 2 reduction for stress concentration)
    "rho_s" : 1700.0,  # [kg/m**3] Volumic mass of aluminum
    "rho_air" : 1.18,  # [kg/m**3] Air density
    "ND_max" : 105000.0 / 60.0 * 0.0254,  # [Hz.m] Max speed limit (N.D max) for APC MR propellers
    "D_pro_ref" : 11.0 * 0.0254,  # [m] Reference propeller diameter
    "M_pro_ref" : 0.53 * 0.0283,  # [kg] Reference propeller mass
}

# Design variables initial values 

design_variables_default = {
    "beta_pro": 0.33,  # pitch/diameter ratio of the propeller
    "k_os": 3.2,  # over sizing coefficient on the load mass
    "k_ND": 1.2,  # slow down propeller coef : ND = NDmax / k_ND
    "k_mot": 1.0,  # over sizing coefficient on the motor torque
    "k_speed_mot": 1.2,  # adaption winding coef on the motor speed
    "k_mb": 1.0,  # ratio battery/load mass
    "k_vb": 1.0,  # oversizing coefficient for voltage evaluation
    "k_D": 0.5,  # aspect ratio D_in/D_out for the beam of the frame
}

Similar to the previous notebook, we define the models to include in the problem as well as the optimization formulation.

In [3]:
prob = om.Problem()

group = om.Group()
group.add_subsystem("scenarios", Scenarios(), promotes=["*"])
group.add_subsystem("propeller", Propeller(), promotes=["*"])
group.add_subsystem("motor", Motor(), promotes=["*"])
group.add_subsystem("battery", Battery(), promotes=["*"])
group.add_subsystem("esc", ESC(), promotes=["*"])
group.add_subsystem("frame", Frame(), promotes=["*"])
group.add_subsystem("objectives", Objectives(), promotes=["*"])
group.add_subsystem("constraints", Constraints(), promotes=["*"])


prob.model = group

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options["optimizer"] = "SLSQP"
prob.driver.options["maxiter"] = 100
prob.driver.options["tol"] = 1e-8

prob.model.add_design_var("beta_pro", lower=0.3, upper=0.6)
prob.model.add_design_var("k_os", lower=1.0, upper=10.0)
prob.model.add_design_var("k_ND", lower=1.0, upper=10.0)
prob.model.add_design_var("k_mot", lower=1.0, upper=10.0)
prob.model.add_design_var("k_speed_mot", lower=1.0, upper=10.0)
prob.model.add_design_var("k_mb", lower=0.1, upper=10.0)
prob.model.add_design_var("k_vb", lower=1.0, upper=10.0)
prob.model.add_design_var("k_D", lower=0.01, upper=0.99)

prob.model.add_constraint("cons_1", lower=0.)
prob.model.add_constraint("cons_2", lower=0.)
prob.model.add_constraint("cons_3", lower=0.)
prob.model.add_constraint("cons_4", lower=0.)
prob.model.add_constraint("cons_5", lower=0.)
prob.model.add_constraint("cons_6", lower=0.)
prob.model.add_constraint('cons_7', lower=0.)

# prob.model.add_objective('t_hov', scaler=-0.1)
prob.model.add_objective("M_total_real", scaler=0.1)

prob.setup()

Now we set the inputs based on the values provided by the dictionnaries.

In [4]:
# Setup initial values
for name, value in {**specifications_default, **design_assumptions_default, **design_variables_default}.items():
    prob.set_val(name, value)

optimization_unsuccessful = prob.run_driver()

/opt/homebrew/Caskroom/miniforge/base/envs/sizing_course39/lib/python3.9/site-packages/scipy/optimize/_optimize.py:284: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: 10.377302308969696
            Iterations: 11
            Function evaluations: 12
            Gradient evaluations: 11
Optimization Complete
-----------------------------------


Please note that if the optimization is unsuccessful `optimization_unsuccessful = True`. This could be useful.

We can check the details of the optimization results:

In [5]:
results = prob.list_problem_vars(print_arrays=True,
                       desvar_opts=['lower', 'upper','min', 'max'],
                       cons_opts=['lower', 'upper', 'equals', 'min', 'max'],
                       objs_opts=['scaler'])

----------------
Design Variables
----------------
name         val           size  lower  upper  min         max         
-----------  ------------  ----  -----  -----  ----------  ---------- 
beta_pro     [0.3]         1     0.3    0.6    0.3         0.3         
k_os         [2.07546046]  1     1.0    10.0   2.07546046  2.07546046  
k_ND         [1.25998295]  1     1.0    10.0   1.25998295  1.25998295  
k_mot        [1.02941176]  1     1.0    10.0   1.02941176  1.02941176  
k_speed_mot  [1.26195253]  1     1.0    10.0   1.26195253  1.26195253  
k_mb         [0.64317582]  1     0.1    10.0   0.64317582  0.64317582  
k_vb         [1.]          1     1.0    10.0   1.0         1.0         
k_D          [0.99]        1     0.01   0.99   0.99        0.99        

-----------
Constraints
-----------
name    val                size  lower  upper  equals  min                max                
------  -----------------  ----  -----  -----  ------  -----------------  ----------------- 
cons_1

And check the details of the final inputs:

In [6]:
inputs = prob.model.list_inputs(prom_name=True)

70 Input(s) in 'model'

varname          val                prom_name    
---------------  -----------------  -------------
scenarios
  k_os           [2.07546046]       k_os         
  M_pay          [50.]              M_pay        
  N_pro_arm      [1.]               N_pro_arm    
  N_arm          [4.]               N_arm        
  a_to           [2.4525]           a_to         
propeller
  beta_pro       [0.3]              beta_pro     
  F_pro_to       [318.12917389]     F_pro_to     
  rho_air        [1.18]             rho_air      
  ND_max         [44.45]            ND_max       
  k_ND           [1.25998295]       k_ND         
  M_pro_ref      [0.014999]         M_pro_ref    
  D_pro_ref      [0.2794]           D_pro_ref    
  F_pro_hov      [254.50333911]     F_pro_hov    
motor
  k_mot          [1.02941176]       k_mot        
  T_pro_hov      [20.72734165]      T_pro_hov    
  k_vb           [1.]               k_vb         
  P_pro_to       [3616.45874542]    P_pro_to     


And check the details of the final outputs:

In [7]:
outputs = prob.model.list_outputs()

54 Explicit Output(s) in 'model'

varname          val                  prom_name    
---------------  -------------------  -------------
scenarios
  M_total        [103.77302308]       M_total      
  N_pro          [4.]                 N_pro        
  F_pro_hov      [254.50333911]       F_pro_hov    
  F_pro_to       [318.12917389]       F_pro_to     
propeller
  C_t            [0.0859]             C_t          
  C_p            [0.02768]            C_p          
  D_pro          [1.58802402]         D_pro        
  n_pro_to       [22.21518979]        n_pro_to     
  Omega_pro_to   [139.5821541]        Omega_pro_to 
  M_pro          [0.48453331]         M_pro        
  P_pro_to       [3616.45874542]      P_pro_to     
  T_pro_to       [25.90917706]        T_pro_to     
  n_pro_hov      [19.8698698]         n_pro_hov    
  Omega_pro_hov  [124.846074]         Omega_pro_hov
  P_pro_hov      [2587.72722963]      P_pro_hov    
  T_pro_hov      [20.72734165]        T_pro_hov    
motor
  T_

Now feel free to copy/duplicate and modify peaces of the previosu code to answer the different exercices.

```{note}
Please highlight during the exercice the design space where no physical solutions are found (if relevant).
```

## Determining the physical limits of the design space

### Payload
```{exercise}
:label: exercise_max_payload

Determine with the default requirements what is the payload value where the UAV design optimization does not find a solution?
```


In [8]:
# To be completed 

Please comment...

### Take-off acceleration
```{exercise}
:label: exercise_max_acceleration

Determine with the default requirements what is the take-off acceleration value where the UAV design optimization does not find a solution?
```

In [9]:
# To be completed 

Please comment...

### Autonomy
```{exercise}
:label: exercise_max_acceleration

Determine with the default requirements what is the autonomy value where the UAV design optimization does not find a solution?
```

In [10]:
# To be completed 

Please comment...

## UAV total mass ($M_{total,real}$)
```{warning}
From now on, deactivate the constraint on the MTOW.
```


### $M_{total,real}$ vs Autonomy
```{exercise}
:label: exercise_max_acceleration


With the default requirements, plot the $M_{total,real}$ for autonomy requirements in the range of [10 mins to 50 mins]
```

In [11]:
# To be completed 

Please comment...

### $M_{total,real}$ vs Payload
```{exercise}
:label: exercise_max_acceleration


With the default requirements, plot the $M_{total,real}$ for payload requirements in the range of [1 kg to 150 kg].
```

In [12]:
# To be completed 

Please comment...

### Autonomy vs Payload
```{exercise}
:label: exercise_max_acceleration


With the default requirements, plot the possible automony (if any) for payload requirements in the range of [1 kg to 150 kg]
```

In [13]:
# To be completed 

Please comment...

## Model modifications

### Motor reference component
```{exercise}
:label: exercise_max_acceleration


Replace the reference motor with the following motor [EMRAX 188 High Voltage ambient
air cooled](https://emrax.com/wp-content/uploads/2022/11/EMRAX_188_datasheet_A00.pdf) and provide the new autonomy of the UAV. Create a new `ExplicitComponent` `MotorEMRAX` if necessary.
```

In [14]:
# To be completed 

Please comment...

### Battery reference component
```{exercise}
:label: exercise_max_acceleration


We have not choice but to use the reference battery pack. What is the new value of the payload to maintain the other default requirements?
```

In [15]:
# To be completed 

Please comment...

## Your turn (Bonus)


```{exercise}
:label: exercise_max_acceleration


Propose your own analysis using the UAV sizing code.
```

In [16]:
# To be completed 

Please comment...